In [19]:
# Imports
from datetime import datetime, timedelta, timezone
import dateutil
import pytz
import zoneinfo

import pendulum
import arrow
import whenever

In [25]:
# Defining test cases:

"""
0) Add 1 month to January 31st or June 31st. -> Next month will have fewer days
1) Add 30 days, 1 year, or 4 years to a date. -> Changing years can be weird because of leap years?
2) Add and then subtract time intervals. Order matters? -> Yes, in some cases. Look back at (1).
3) Multiply or divide of time intervals. -> Weird when considering leap days. The new time interval depends on the context.
4) Handle datetime arithmetic when database information updates. -> Database information can change in an infinite number of ways.
5) Leap seconds during leap days? -> Can affect some calculations I guess.
6) Add one day to 1:30 AM of the day before a leap day. Do you get fold=0 or fold=1? "Landing into a timezone shift"
7) Add both a date and a time in one operation. Which should come first? -> In any case, keep them separate. In addition, you run into problem (7).
8) Does anything strange happen if you say "Alice went from time x to y. Now Bob moves forward that timedelta. Are Alice and Bob the same distance away?"
9) Add a day to day x at 11PM. During the DST shift, an hour is skipped. Are you now in x + 1 in x + 2? What about the reverse?
"""

# Defining test cases:

du_tzNYC = dateutil.tz.gettz("America/New_York")

tcs = [
        datetime(2024, 6, 7, 23), # 0: Normal. Should be 18 hours.
        datetime(2022, 3, 12, 23), # 1: Skip an hour that night. Should be 17 hours.
        datetime(2022, 11, 5, 23), # 2: Go back an hour that night. Should be 19 hours.
        datetime(2022, 3, 13, 1), # 3: Skip an hour this morning. Should be 39 hours.
        datetime(2022, 11, 6, 1, fold=0), # 4: Before going back an hour that morning. Should be 41 hours.
        datetime(2022, 11, 6, 1, 30, fold=0), # 5: Within DST hour 1st time. Should be 40 hours 30 minutes.
        datetime(2022, 11, 6, 1, 30, fold=1), # 6: Within DST hour 2nd time. Should be 39 hours 30 minutes.
        datetime(2022, 11, 5, 2, 30), # 8: Tm at this time is during DST 2nd time. Should be 39 hours 30 minutes.
        datetime(2022, 11, 5, 1, 30) # 7: Tm at this time is during DST 1st time. Should be 40 hours 30 minutes.
]

tcs_pendulum = [
        pendulum.datetime(2024, 6, 7, 23, tz="America/New_York"), # 0: Normal. Should be 18 hours.
        pendulum.datetime(2022, 3, 12, 23, tz="America/New_York"), # 1: Skip an hour that night. Should be 17 hours.
        pendulum.datetime(2022, 11, 5, 23, tz="America/New_York"), # 2: Go back an hour that night. Should be 19 hours.
        pendulum.datetime(2022, 3, 13, 1, tz="America/New_York"), # 3: Skip an hour this morning. Should be 39 hours.
        pendulum.datetime(2022, 11, 6, 1, dst_rule=pendulum.PRE_TRANSITION, tz="America/New_York"), # 4: Before going back an hour that morning. Should be 41 hours.
        pendulum.datetime(2022, 11, 6, 1, 30, dst_rule=pendulum.PRE_TRANSITION, tz="America/New_York"), # 5: Within DST hour 1st time. Should be 40 hours 30 minutes.
        pendulum.datetime(2022, 11, 6, 1, 30, dst_rule=pendulum.POST_TRANSITION, tz="America/New_York"), # 6: Within DST hour 2nd time. Should be 39 hours 30 minutes.
        pendulum.datetime(2022, 11, 5, 1, 30, tz="America/New_York"), # 7: Tm at this time is during DST 1st time. Should be 40 hours 30 minutes.
        pendulum.datetime(2022, 11, 5, 2, 30, tz="America/New_York") # 8: Tm at this time is during DST 2nd time. Should be 39 hours 30 minutes.
]

tcs_arrow = [
        arrow.get(2024, 6, 7, 23, tzinfo=du_tzNYC), # 0: Normal. Should be 18 hours.
        arrow.get(2022, 3, 12, 23, tzinfo=du_tzNYC), # 1: Skip an hour that night. Should be 17 hours.
        arrow.get(2022, 11, 5, 23, tzinfo=du_tzNYC), # 2: Go back an hour that night. Should be 19 hours.
        arrow.get(2022, 3, 13, 1, tzinfo=du_tzNYC), # 3: Skip an hour this morning. Should be 39 hours.
        arrow.get(2022, 11, 6, 1, tzinfo=du_tzNYC, fold=0), # 4: Before going back an hour that morning. Should be 41 hours.
        arrow.get(2022, 11, 6, 1, 30, tzinfo=du_tzNYC, fold=0), # 5: Within DST hour 1st time. Should be 40 hours 30 minutes.
        arrow.get(2022, 11, 6, 1, 30, tzinfo=du_tzNYC, fold=1), # 6: Within DST hour 2nd time. Should be 39 hours 30 minutes.
        arrow.get(2022, 11, 5, 1, 30, tzinfo=du_tzNYC), # 7: Tm at this time is during DST 1st time. Should be 40 hours 30 minutes.
        arrow.get(2022, 11, 5, 2, 30, tzinfo=du_tzNYC) # 8: Tm at this time is during DST 2nd time. Should be 39 hours 30 minutes.
]

tcs_whenever = [
        whenever.ZonedDateTime(2024, 6, 7, 23, tz="America/New_York"), # 0: Normal. Should be 18 hours.
        whenever.ZonedDateTime(2022, 3, 12, 23, tz="America/New_York"), # 1: Skip an hour that night. Should be 17 hours.
        whenever.ZonedDateTime(2022, 11, 5, 23, tz="America/New_York"), # 2: Go back an hour that night. Should be 19 hours.
        whenever.ZonedDateTime(2022, 3, 13, 1, tz="America/New_York"), # 3: Skip an hour this morning. Should be 39 hours.
        whenever.ZonedDateTime(2022, 11, 6, 1, tz="America/New_York", disambiguate="earlier"), # 4: Before going back an hour that morning. Should be 41 hours.
        whenever.ZonedDateTime(2022, 11, 6, 1, 30, tz="America/New_York", disambiguate="earlier"), # 5: Within DST hour 1st time. Should be 40 hours 30 minutes.
        whenever.ZonedDateTime(2022, 11, 6, 1, 30, tz="America/New_York", disambiguate="later"), # 6: Within DST hour 2nd time. Should be 39 hours 30 minutes.
        whenever.ZonedDateTime(2022, 11, 5, 1, 30, tz="America/New_York"), # 7: Tm at this time is during DST 1st time. Should be 40 hours 30 minutes.
        whenever.ZonedDateTime(2022, 11, 5, 2, 30, tz="America/New_York") # 8: Tm at this time is during DST 2nd time. Should be 39 hours 30 minutes.
]

In [101]:
def compare_outputs(outputs):
    timezone_keys = list(outputs.keys())
    num_test_cases = len(next(iter(outputs.values())))
    test_cases_with_differences = []

    for k in range(num_test_cases):
        output_set = set(outputs[tzp][k] for tzp in timezone_keys)
        if len(output_set) > 1:
            test_cases_with_differences.append(k)

    return test_cases_with_differences

In [102]:
def get_calculations(tc_num, dt, tzNYC, tzp) -> str:

        dt = tzNYC.localize(dt) if tzp == "pytz" else dt.astimezone(tzNYC)

        dt2 = dt + timedelta(days=1)
        dt3 = dt - timedelta(days=1)

        # both of the following implementations have the same effect
        # 
        # this problem can likely only be solved using \
        # careful manipulation on our parts
        # 
        # if (tzp == "pytz"):
        #         dt2 = dt2.astimezone(tzNYC)
        #         dt3 = dt3.astimezone(tzNYC)
                
        if (tzp == "pytz"):
                dt2 = tzNYC.normalize(dt2)
                dt3 = tzNYC.normalize(dt3)
        
        ret = ""
        ret += f"-1d: {dt2}\t"
        ret += f"00d: {dt}\t"
        ret += f"+1d: {dt3}\t"
        return ret

def get_calculations_module(tc_num, dt, tzp) -> str:

        dt2 = None
        dt3 = None

        if tzp == "Pendulum":
                dt2 = dt.add(days=1)
                dt3 = dt.subtract(days=1)
        if tzp == "Arrow":
                dt2 = dt + timedelta(days=1)
                dt3 = dt - timedelta(days=1)
        if tzp == "Whenever":
                dt2 = dt + whenever.days(1)
                dt3 = dt - whenever.days(1)

        str1 = str(dt2)
        str2 = str(dt)
        str3 = str(dt3)

        str1 = str1[:10] + " " + str1[11:]
        str2 = str2[:10] + " " + str2[11:]
        str3 = str3[:10] + " " + str3[11:]

        str1 = str1[:25]
        str2 = str2[:25]
        str3 = str3[:25]

        ret = ""
        ret += f"-1d: {str1}\t"
        ret += f"00d: {str2}\t"
        ret += f"+1d: {str3}\t"
        return ret


du_tzNYC = dateutil.tz.gettz("America/New_York")
du_tzUTC = dateutil.tz.gettz("UTC")
pytz_tzNYC = pytz.timezone("America/New_York")
pytz_tzUTC = pytz.timezone("UTC")
zi_tzNYC = zoneinfo.ZoneInfo("America/New_York")
zi_tzUTC = zoneinfo.ZoneInfo("UTC")

timezones = {
                "pytz":(pytz_tzNYC, pytz_tzUTC),
                "dateutil.tz":(du_tzNYC, du_tzUTC),
                "zoneinfo":(zi_tzNYC, zi_tzUTC)
             }


modules = ["Pendulum", "Arrow", "Whenever"]
tcs_modules = {"Pendulum": tcs_pendulum, "Arrow": tcs_arrow, "Whenever": tcs_whenever}

outputs_add_sub = {}

for tzp in timezones.keys():
        outputs_0 = []
        for index, tc in enumerate(tcs):
                outputs_0.append(get_calculations(index, tc, timezones[tzp][0], tzp))
        outputs_add_sub[tzp] = outputs_0

for tzp in modules:
        outputs_0 = []
        for index, tc in enumerate(tcs_modules[tzp]):
                outputs_0.append(get_calculations_module(index, tc, tzp))
        outputs_add_sub[tzp] = outputs_0

differences = compare_outputs(outputs_add_sub)
if differences:
    for index in differences:
        print(f"Test case {index} has differences:")
        for idx, tzp in enumerate(outputs_add_sub):
            print(f"{idx}: {outputs_add_sub[tzp][index]}")
else:
    print("No differences found among the timezone providers.")


Test case 1 has differences:
0: -1d: 2022-03-14 00:00:00-04:00	00d: 2022-03-12 23:00:00-05:00	+1d: 2022-03-11 23:00:00-05:00	
1: -1d: 2022-03-13 23:00:00-04:00	00d: 2022-03-12 23:00:00-05:00	+1d: 2022-03-11 23:00:00-05:00	
2: -1d: 2022-03-13 23:00:00-04:00	00d: 2022-03-12 23:00:00-05:00	+1d: 2022-03-11 23:00:00-05:00	
3: -1d: 2022-03-13 23:00:00-04:00	00d: 2022-03-12 23:00:00-05:00	+1d: 2022-03-11 23:00:00-05:00	
4: -1d: 2022-03-13 23:00:00-04:00	00d: 2022-03-12 23:00:00-05:00	+1d: 2022-03-11 23:00:00-05:00	
5: -1d: 2022-03-13 23:00:00-04:00	00d: 2022-03-12 23:00:00-05:00	+1d: 2022-03-11 23:00:00-05:00	
Test case 2 has differences:
0: -1d: 2022-11-06 22:00:00-05:00	00d: 2022-11-05 23:00:00-04:00	+1d: 2022-11-04 23:00:00-04:00	
1: -1d: 2022-11-06 23:00:00-05:00	00d: 2022-11-05 23:00:00-04:00	+1d: 2022-11-04 23:00:00-04:00	
2: -1d: 2022-11-06 23:00:00-05:00	00d: 2022-11-05 23:00:00-04:00	+1d: 2022-11-04 23:00:00-04:00	
3: -1d: 2022-11-06 23:00:00-05:00	00d: 2022-11-05 23:00:00-04:00	+1d:

Test case 1 has differences:

0: -1d: 2022-03-14 00:00:00-04:00	00d: 2022-03-12 23:00:00-05:00	+1d: 2022-03-11 23:00:00-05:00	

1: -1d: 2022-03-13 23:00:00-04:00	00d: 2022-03-12 23:00:00-05:00	+1d: 2022-03-11 23:00:00-05:00	

2: -1d: 2022-03-13 23:00:00-04:00	00d: 2022-03-12 23:00:00-05:00	+1d: 2022-03-11 23:00:00-05:00	

3: -1d: 2022-03-13 23:00:00-04:00	00d: 2022-03-12 23:00:00-05:00	+1d: 2022-03-11 23:00:00-05:00	

4: -1d: 2022-03-13 23:00:00-04:00	00d: 2022-03-12 23:00:00-05:00	+1d: 2022-03-11 23:00:00-05:00	

5: -1d: 2022-03-13 23:00:00-04:00	00d: 2022-03-12 23:00:00-05:00	+1d: 2022-03-11 23:00:00-05:00	

Test case 2 has differences:

0: -1d: 2022-11-06 22:00:00-05:00	00d: 2022-11-05 23:00:00-04:00	+1d: 2022-11-04 23:00:00-04:00	

1: -1d: 2022-11-06 23:00:00-05:00	00d: 2022-11-05 23:00:00-04:00	+1d: 2022-11-04 23:00:00-04:00	

2: -1d: 2022-11-06 23:00:00-05:00	00d: 2022-11-05 23:00:00-04:00	+1d: 2022-11-04 23:00:00-04:00	

3: -1d: 2022-11-06 23:00:00-05:00	00d: 2022-11-05 23:00:00-04:00	+1d: 2022-11-04 23:00:00-04:00	

4: -1d: 2022-11-06 23:00:00-05:00	00d: 2022-11-05 23:00:00-04:00	+1d: 2022-11-04 23:00:00-04:00	

5: -1d: 2022-11-06 23:00:00-05:00	00d: 2022-11-05 23:00:00-04:00	+1d: 2022-11-04 23:00:00-04:00	

Test case 3 has differences:

0: -1d: 2022-03-14 02:00:00-04:00	00d: 2022-03-13 01:00:00-05:00	+1d: 2022-03-12 01:00:00-05:00	

1: -1d: 2022-03-14 01:00:00-04:00	00d: 2022-03-13 01:00:00-05:00	+1d: 2022-03-12 01:00:00-05:00	

2: -1d: 2022-03-14 01:00:00-04:00	00d: 2022-03-13 01:00:00-05:00	+1d: 2022-03-12 01:00:00-05:00	

3: -1d: 2022-03-14 01:00:00-04:00	00d: 2022-03-13 01:00:00-05:00	+1d: 2022-03-12 01:00:00-05:00	

4: -1d: 2022-03-14 01:00:00-04:00	00d: 2022-03-13 01:00:00-05:00	+1d: 2022-03-12 01:00:00-05:00	

5: -1d: 2022-03-14 01:00:00-04:00	00d: 2022-03-13 01:00:00-05:00	+1d: 2022-03-12 01:00:00-05:00	

Test case 4 has differences:

0: -1d: 2022-11-07 01:00:00-05:00	00d: 2022-11-06 01:00:00-05:00	+1d: 2022-11-05 02:00:00-04:00	

1: -1d: 2022-11-07 01:00:00-05:00	00d: 2022-11-06 01:00:00-04:00	+1d: 2022-11-05 01:00:00-04:00	

2: -1d: 2022-11-07 01:00:00-05:00	00d: 2022-11-06 01:00:00-04:00	+1d: 2022-11-05 01:00:00-04:00	

3: -1d: 2022-11-07 01:00:00-05:00	00d: 2022-11-06 01:00:00-04:00	+1d: 2022-11-05 01:00:00-04:00	

4: -1d: 2022-11-07 01:00:00-05:00	00d: 2022-11-06 01:00:00-04:00	+1d: 2022-11-05 01:00:00-04:00	

5: -1d: 2022-11-07 01:00:00-05:00	00d: 2022-11-06 01:00:00-04:00	+1d: 2022-11-05 01:00:00-04:00	

Test case 5 has differences:

0: -1d: 2022-11-07 01:30:00-05:00	00d: 2022-11-06 01:30:00-05:00	+1d: 2022-11-05 02:30:00-04:00	

1: -1d: 2022-11-07 01:30:00-05:00	00d: 2022-11-06 01:30:00-04:00	+1d: 2022-11-05 01:30:00-04:00	

2: -1d: 2022-11-07 01:30:00-05:00	00d: 2022-11-06 01:30:00-04:00	+1d: 2022-11-05 01:30:00-04:00	

3: -1d: 2022-11-07 01:30:00-05:00	00d: 2022-11-06 01:30:00-04:00	+1d: 2022-11-05 01:30:00-04:00	

4: -1d: 2022-11-07 01:30:00-05:00	00d: 2022-11-06 01:30:00-04:00	+1d: 2022-11-05 01:30:00-04:00	

5: -1d: 2022-11-07 01:30:00-05:00	00d: 2022-11-06 01:30:00-04:00	+1d: 2022-11-05 01:30:00-04:00	

Test case 6 has differences:

0: -1d: 2022-11-07 01:30:00-05:00	00d: 2022-11-06 01:30:00-05:00	+1d: 2022-11-05 02:30:00-04:00	

1: -1d: 2022-11-07 01:30:00-05:00	00d: 2022-11-06 01:30:00-05:00	+1d: 2022-11-05 01:30:00-04:00	

2: -1d: 2022-11-07 01:30:00-05:00	00d: 2022-11-06 01:30:00-05:00	+1d: 2022-11-05 01:30:00-04:00	

3: -1d: 2022-11-07 01:30:00-05:00	00d: 2022-11-06 01:30:00-05:00	+1d: 2022-11-05 01:30:00-04:00	

4: -1d: 2022-11-07 01:30:00-05:00	00d: 2022-11-06 01:30:00-05:00	+1d: 2022-11-05 01:30:00-04:00	

5: -1d: 2022-11-07 01:30:00-05:00	00d: 2022-11-06 01:30:00-05:00	+1d: 2022-11-05 01:30:00-04:00	

Test case 7 has differences:

0: -1d: 2022-11-06 01:30:00-05:00	00d: 2022-11-05 02:30:00-04:00	+1d: 2022-11-04 02:30:00-04:00	

1: -1d: 2022-11-06 02:30:00-05:00	00d: 2022-11-05 02:30:00-04:00	+1d: 2022-11-04 02:30:00-04:00	

2: -1d: 2022-11-06 02:30:00-05:00	00d: 2022-11-05 02:30:00-04:00	+1d: 2022-11-04 02:30:00-04:00	

3: -1d: 2022-11-06 01:30:00-05:00	00d: 2022-11-05 01:30:00-04:00	+1d: 2022-11-04 01:30:00-04:00	

4: -1d: 2022-11-06 01:30:00-04:00	00d: 2022-11-05 01:30:00-04:00	+1d: 2022-11-04 01:30:00-04:00	

5: -1d: 2022-11-06 01:30:00-04:00	00d: 2022-11-05 01:30:00-04:00	+1d: 2022-11-04 01:30:00-04:00	

Test case 8 has differences:

0: -1d: 2022-11-06 01:30:00-04:00	00d: 2022-11-05 01:30:00-04:00	+1d: 2022-11-04 01:30:00-04:00	

1: -1d: 2022-11-06 01:30:00-04:00	00d: 2022-11-05 01:30:00-04:00	+1d: 2022-11-04 01:30:00-04:00	

2: -1d: 2022-11-06 01:30:00-04:00	00d: 2022-11-05 01:30:00-04:00	+1d: 2022-11-04 01:30:00-04:00	

3: -1d: 2022-11-06 02:30:00-05:00	00d: 2022-11-05 02:30:00-04:00	+1d: 2022-11-04 02:30:00-04:00	

4: -1d: 2022-11-06 02:30:00-05:00	00d: 2022-11-05 02:30:00-04:00	+1d: 2022-11-04 02:30:00-04:00	

5: -1d: 2022-11-06 02:30:00-05:00	00d: 2022-11-05 02:30:00-04:00	+1d: 2022-11-04 02:30:00-04:00	

**What we find**

Notice that pytz is different from most of the other implementations

We must also consider if maybe pytz is actually the correct one

Additionally, test cases 7 and 8 differ between the default datetime library timezone providers and the modules

In [207]:
du_tzNYC = dateutil.tz.gettz("America/New_York")
du_tzLA = dateutil.tz.gettz("America/Los_Angeles")
du_tzUTC = dateutil.tz.gettz("UTC")

tcs_equality = [
        ("Same date, no TZs", datetime(2024, 6, 7, 23), datetime(2024, 6, 7, 23)),
        ("Same date, one has TZ", datetime(2024, 6, 7, 23), datetime(2024, 6, 7, 23, tzinfo=du_tzNYC)),
        ("Same date and TZ", datetime(2024, 6, 7, 23, tzinfo=du_tzNYC), datetime(2024, 6, 7, 23, tzinfo=du_tzNYC)),
        ("Same date, diff TZ", datetime(2024, 6, 7, 23, tzinfo=du_tzNYC), datetime(2024, 6, 7, 23, tzinfo=du_tzUTC)),
        ("Same UTC, diff TZ", datetime(2024, 6, 7, 23, tzinfo=du_tzUTC).astimezone(du_tzNYC), datetime(2024, 6, 7, 23, tzinfo=du_tzUTC).astimezone(du_tzLA)),
        ("Same date, diff fold", datetime(2022, 11, 6, 1, 30, tzinfo=du_tzNYC, fold=0), datetime(2022, 11, 6, 1, 30, tzinfo=du_tzNYC, fold=1))
]

tcs_equality_pendulum = [
        ("Same date, no TZs", pendulum.datetime(2024, 6, 7, 23), pendulum.datetime(2024, 6, 7, 23)),
        ("Same date, one has TZ", pendulum.datetime(2024, 6, 7, 23), pendulum.datetime(2024, 6, 7, 23, tz="America/New_York")),
        ("Same date and TZ", pendulum.datetime(2024, 6, 7, 23, tz="America/New_York"), pendulum.datetime(2024, 6, 7, 23, tz="America/New_York")),
        ("Same date, diff TZ", pendulum.datetime(2024, 6, 7, 23, tz="America/New_York"), pendulum.datetime(2024, 6, 7, 23, tz="UTC")),
        ("Same UTC, diff TZ", pendulum.datetime(2024, 6, 7, 23, tz="UTC").in_timezone(du_tzNYC), pendulum.datetime(2024, 6, 7, 23, tz="UTC").in_timezone(du_tzLA)),
        ("Same date, diff fold", pendulum.datetime(2022, 11, 6, 1, 30, tz="America/New_York", dst_rule=pendulum.PRE_TRANSITION), pendulum.datetime(2022, 11, 6, 1, 30, tz="America/New_York", dst_rule=pendulum.POST_TRANSITION))
]

tcs_equality_arrow = [
        ("Same date, no TZs", arrow.get(2024, 6, 7, 23), arrow.get(2024, 6, 7, 23)),
        ("Same date, one has TZ", arrow.get(2024, 6, 7, 23), arrow.get(2024, 6, 7, 23, tzinfo=du_tzNYC)),
        ("Same date and TZ", arrow.get(2024, 6, 7, 23, tzinfo=du_tzNYC), arrow.get(2024, 6, 7, 23, tzinfo=du_tzNYC)),
        ("Same date, diff TZ", arrow.get(2024, 6, 7, 23, tzinfo=du_tzNYC), arrow.get(2024, 6, 7, 23, tzinfo=du_tzUTC)),
        ("Same UTC, diff TZ", arrow.get(2024, 6, 7, 23, tzinfo=du_tzUTC).astimezone(du_tzNYC), arrow.get(2024, 6, 7, 23, tzinfo=du_tzUTC).astimezone(du_tzLA)),
        ("Same date, diff fold", arrow.get(2022, 11, 6, 1, 30, tzinfo=du_tzNYC, fold=0), arrow.get(2022, 11, 6, 1, 30, tzinfo=du_tzNYC, fold=1))
]

tcs_equality_whenever = [
        ("Same date, no TZs", whenever.NaiveDateTime(2024, 6, 7, 23), whenever.NaiveDateTime(2024, 6, 7, 23)),
        ("Same date, one has TZ", whenever.NaiveDateTime(2024, 6, 7, 23), whenever.ZonedDateTime(2024, 6, 7, 23, tz="America/New_York")),
        ("Same date and TZ", whenever.ZonedDateTime(2024, 6, 7, 23, tz="America/New_York"), whenever.ZonedDateTime(2024, 6, 7, 23, tz="America/New_York")),
        ("Same date, diff TZ", whenever.ZonedDateTime(2024, 6, 7, 23, tz="America/New_York"), whenever.ZonedDateTime(2024, 6, 7, 23, tz="UTC")),
        ("Same UTC, diff TZ", whenever.ZonedDateTime(2024, 6, 7, 23, tz="UTC").as_zoned("America/New_York"), whenever.ZonedDateTime(2024, 6, 7, 23, tz="UTC").as_zoned("America/Los_Angeles")),
        ("Same date, diff fold", whenever.ZonedDateTime(2022, 11, 6, 1, 30, tz="America/New_York", disambiguate="earlier"), whenever.ZonedDateTime(2022, 11, 6, 1, 30, tz="America/New_York", disambiguate="later"))
]

modules = ["Pnm", "Arw", "Wvr"]
tcs_equality_modules = {"Pnm": tcs_equality_pendulum, "Arw": tcs_equality_arrow, "Wvr": tcs_equality_whenever}

outputs_equality = {}

outputs_equality["DTm"] = []
for index, tc in enumerate(tcs_equality):
        outputs_equality["DTm"].append(str(tc[1] == tc[2]))

for tzp in modules:
        outputs_0 = []
        for index, tc in enumerate(tcs_equality_modules[tzp]):
                outputs_0.append(str(tc[1] == tc[2]))
        outputs_equality[tzp] = outputs_0

differences = compare_outputs(outputs_equality)
if differences:
    for index in differences:
        print(f"Test case {index} ({tcs_equality[index][0]}) has differences:")
        for idx, tzp in enumerate(outputs_equality):
            print(f"{tzp}: {outputs_equality[tzp][index]}")
else:
    print("No differences found among the timezone providers.")


Test case 5 (Same date, diff fold) has differences:
DTm: True
Pnm: True
Arw: True
Wvr: False


Test case 5 (Same date, diff fold) has differences:

DTm: True

Pnm: True

Arw: True

Wvr: False

In [204]:
# I'm going to assume that all these times are in NYC
def __new_eq__(dt1, dt2) -> bool:
        if (type(dt1) != type(dt2)):
                return False
        if type(dt1) == datetime:
                if dt1.fold != dt2.fold: return False
                return dt1 == dt2
        elif type(dt1) == arrow.arrow.Arrow:
                if dt1.fold != dt2.fold:
                        return False
                return dt1 == dt2
        elif type(dt1) == pendulum.DateTime:
                if dt1.in_timezone(du_tzUTC) != dt2.in_timezone(du_tzUTC): return False
                return dt1 == dt2
        elif type(dt1) in {whenever.ZonedDateTime, whenever.NaiveDateTime}:
                return dt1 == dt2
        else:
                return False

In [206]:
outputs_equality_new = {}

outputs_equality_new["DTm"] = []
for index, tc in enumerate(tcs_equality):
        outputs_equality_new["DTm"].append(str(__new_eq__(tc[1], tc[2])))

for tzp in modules:
        outputs_0 = []
        for index, tc in enumerate(tcs_equality_modules[tzp]):
                outputs_0.append(str(__new_eq__(tc[1], tc[2])))
        outputs_equality_new[tzp] = outputs_0

differences = compare_outputs(outputs_equality_new)
if differences:
    for index in differences:
        print(f"Test case {index} ({tcs_equality[index][0]}) has differences:")
        for idx, tzp in enumerate(outputs_equality_new):
            print(f"{tzp}: {outputs_equality_new[tzp][index]}")
else:
    print("No differences found among the timezone providers.")


No differences found among the timezone providers.
